# Whisper-tiny test 2
### test1과 다른 점

* training_args = TrainingArguments -> Seq2SeqTrainingArguments
*   learning_rate 5e-5 -> 2e-5
*   num_train_epochs 3 -> 10
* logging_steps 10 -> 20
* save_steps -> save_strategy=epoch
* predict_with_generate=True 추가



##1. 데이터 불러오기

### 라이브러리 설치 및 로드

In [ ]:
!pip install -U transformers datasets torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [ ]:
!pip install -U transformers datasets huggingface_hub evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transfor

### Hugging Face 로그인

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### 라이브러리 임포트

In [ ]:
from datasets import load_dataset, Audio, Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from evaluate import load
import random
import torch

### 데이터셋 스트리밍 로드

In [ ]:
n_train = 200
n_val = 100
n_test = 100

langs = ["ko", "ja", "en", "de"]

In [ ]:
train_data = []
val_data = []
test_data = []

for lang in langs:
    print(f"✅ Loading {lang}...")

    train_data += list(load_dataset(
        "mozilla-foundation/common_voice_16_1", lang,
        split="train",
        streaming=True,
        trust_remote_code=True
    ).take(n_train))

    val_data += list(load_dataset(
        "mozilla-foundation/common_voice_16_1", lang,
        split="validation",
        streaming=True,
        trust_remote_code=True
    ).take(n_val))

    test_data += list(load_dataset(
        "mozilla-foundation/common_voice_16_1", lang,
        split="test",
        streaming=True,
        trust_remote_code=True
    ).take(n_test))


✅ Loading ko...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

common_voice_16_1.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

...

Reading metadata...: 9616it [00:01, 6905.48it/s]
Reading metadata...: 6094it [00:01, 4431.59it/s]
Reading metadata...: 6094it [00:01, 5308.93it/s]


✅ Loading en...


Reading metadata...: 1090061it [01:03, 17131.94it/s]
Reading metadata...: 16390it [00:01, 9928.59it/s]
Reading metadata...: 16390it [00:01, 10020.57it/s]


✅ Loading de...


Reading metadata...: 576207it [00:36, 15901.16it/s]
Reading metadata...: 16173it [00:01, 9995.59it/s]
Reading metadata...: 16173it [00:00, 19677.30it/s]


In [ ]:
# 데이터 섞기(셔플)
random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)

In [ ]:
# Hugging Face Dataset 객체로 변환
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

##2. 데이터 전처리


*   Whisper Tokenizer 준비
*   학습 데이터 포맷 맞추기



### Whisper PRocessor 준비


*   Whisper 모델 사전 학습 체크포인트사용
*   Processor = FeatureExtractor + Tokenizer
* 오디오 -> 입력 특징 추출
* 텍스트 -> 토큰화



In [ ]:
model_checkpoint = "openai/whisper-tiny"

processor = WhisperProcessor.from_pretrained(model_checkpoint)
feature_extractor = processor.feature_extractor
tokenizer = processor.tokenizer

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

### 오디오 컬럼 샘플링레이트에 맞추기

In [ ]:
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

### 전처리 함수 정의


*   오디오 -> Whisper 입력 특징 추출
*   텍스트 문장 -> Whisper 토큰화


*   학습 데이터 준비





In [ ]:
def prepare_dataset(example):
    audio = example["audio"]
    example["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=16000
    ).input_features[0]

    # ✅ 중요: 리스트 상태로 저장 (padding X, tensor 변환 X)
    example["labels"] = processor.tokenizer(
        example["sentence"]
    ).input_ids

    return example

### 전처리 매핑


*   prepare_batch함수를 모든 샘플에 적용
*   필요없는 컬럼 제거



In [ ]:
processed_dataset_train = train_dataset.map(
    prepare_dataset,
    remove_columns=[col for col in train_dataset.column_names if col not in ["locale"]]
)

processed_dataset_val = val_dataset.map(
    prepare_dataset,
    remove_columns=[col for col in val_dataset.column_names if col not in ["locale"]]
)

processed_dataset_test = test_dataset.map(
    prepare_dataset,
    remove_columns=[col for col in test_dataset.column_names if col not in ["locale"]]
)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

### Torch 포맷으로 변환


*   Datasets -> Pytorch Tensor 형태로 변환
*   Trainer가 DataLoader처럼 읽을 수 있게 준비



In [ ]:
processed_dataset_train.set_format(columns=["input_features", "labels", "locale"])
processed_dataset_val.set_format(columns=["input_features", "labels", "locale"])
processed_dataset_test.set_format(columns=["input_features", "labels", "locale"])

##3. Whisper 모델 불러오기

### 모델 불러오기


*   openai/Whisper-tiny 체크포인트에서 모델 로드
*   사전학습된 Whisper모델 -> Fine-tuning 준비 완료



In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### 파라미터 세팅


*   학습 출력 경로 지정
*   배치 사이즈, running rate, epoch수 설정
* fp16 자동 사용(가능한 경우)



In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=10,
    logging_steps=20,
    save_strategy="epoch",
    # evaluation_strategy="epoch", # Removed this argument
    fp16=torch.cuda.is_available(),
    predict_with_generate=True,
    report_to="none"
)

### Trainer 정의


*   학습할 모델, 파라미터 세팅
*   학습용(train) / 검증용(eval) 데이터셋 지정
* Whisper feature extractor -> 입력 전처리 자동 적용



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset_train,
    eval_dataset=processed_dataset_val,
    tokenizer=processor,
    data_collator=data_collator,
)

/tmp/ipython-input-17-3433344463.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### 학습 시작


*   항목 추가
*   항목 추가



In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
20,2.989300
40,1.155600
60,1.007800
80,0.847900
100,0.906800
120,0.471300
140,0.429200
160,0.424600
180,0.407200
200,0.448300


TrainOutput(global_step=1000, training_loss=0.21904065757989882, metrics={'train_runtime': 1233.2766, 'train_samples_per_second': 6.487, 'train_steps_per_second': 0.811, 'total_flos': 1.9695108096e+17, 'train_loss': 0.21904065757989882, 'epoch': 10.0})

### 모델 저장

In [ ]:
trainer.save_model("./whisper-tiny-finetuned2")

### Processor 저장

In [ ]:
# ✅ Whisper Processor (Tokenizer + FeatureExtractor)도 같이 저장
processor.save_pretrained("./whisper-tiny-finetuned2")

[]

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.1606972217559814, 'eval_runtime': 50.1704, 'eval_samples_per_second': 7.973, 'eval_steps_per_second': 0.997, 'epoch': 10.0}


##4. Test

### 언어별로 Validation 데이터 나누기

In [ ]:
# ✅ 각 언어별 검증(validation) 데이터셋 필터링
val_ko = processed_dataset_val.filter(lambda x: x["locale"] == "ko")
val_en = processed_dataset_val.filter(lambda x: x["locale"] == "en")
val_ja = processed_dataset_val.filter(lambda x: x["locale"] == "ja")
val_de = processed_dataset_val.filter(lambda x: x["locale"] == "de")

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

### 언어 리스트와 매핑 딕셔너리 준비

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.7 MB/s eta 0:00:00


In [ ]:
from evaluate import load

lang_datasets = {
    "ko": val_ko,
    "en": val_en,
    "ja": val_ja,
    "de": val_de
}

wer_metric = load("wer")
cer_metric = load("cer")

In [ ]:
lang_scores = {}

for lang, dataset in lang_datasets.items():
    print(f"\n🌍 Language: {lang.upper()}")

    predictions_output = trainer.predict(dataset)
    logits = predictions_output.predictions[0]

    pred_ids = logits.argmax(axis=-1)

    decoded_preds = processor.tokenizer.batch_decode(
        pred_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    decoded_labels = processor.tokenizer.batch_decode(
        predictions_output.label_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )

    # ✅ 예측 문장 출력 (원하는 만큼 출력)
    for pred, label in zip(decoded_preds, decoded_labels):
        print(f"🔹 Prediction: {pred}")
        print(f"🔸 Reference : {label}")
        print("------")

    wer_score = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    cer_score = cer_metric.compute(predictions=decoded_preds, references=decoded_labels)

    lang_scores[lang] = {"WER": wer_score, "CER": cer_score}

    print(f"✅ {lang.upper()} WER: {wer_score:.4f}")
    print(f"✅ {lang.upper()} CER: {cer_score:.4f}")


🌍 Language: KO


🔹 Prediction: 그는 되게게 해�빛이고 공기였어요. 122me2meme2222222222222222!
🔸 Reference : 그는 제게 햇빛이고 공기였어요
------
🔹 Prediction: 옥점의 뒤를 따라 들어오는 남�모��를 양복쟁이는 모념�� 바라보며 머묇모�리�� 타 섰다.!
🔸 Reference : 옥점의 뒤를 따라 들어오는 낯모를 양복쟁이는 모녀를 바라보며 머뭇머뭇 하고 섰다.
------
🔹 Prediction: 덕호는 식�씩 가며 선비의 기에다 입을 대고 이렇게 수� 걸��다.오� 오������!
🔸 Reference : 덕호는 씩씩하며 선비의 귀에다 입을 대고 이렇게 수군거렸다.
------
🔹 Prediction: 다만히 그림이 훌륭하다는 것을 자랑하는 재미�이 모양이다.아아32아아아2222222!
🔸 Reference : 다만 이 그림이 훌륭하다는 것을 자랑하는 셈인 모양이다.
------
🔹 Prediction: 頭다�에게 이루��대。あああありるるるるるるるるる)))))))。。。。。。!
🔸 Reference : 아담에게 이르시되
------
🔹 Prediction: 다시 사람��날 자신이 있는 듯이 가녉품 미소를 띠어 보인다. 2222222332!
🔸 Reference : 다시 살아날 자신이 있는 듯이 가냘픈 미소를 띠어 보인다.
------
🔹 Prediction: � 고�은 소리로 덕호는 이렇게 말하였다. � � � �고고고고5아222222222!
🔸 Reference : 혀 곱은 소리로 덕호는 이렇게 말하였다.
------
🔹 Prediction: 그랗��니 그중 몇 개집을가 싹 돌아아서서 글쎄 감독에게 고의 받쳤구나.22D222!
🔸 Reference : 그랬더니 그 중에 몇 계집애가 싹 돌아서서 글쎄 감독에게 고해 바쳤구나.
------
🔹 Prediction: 새�� 그릇을 받아 들었을 때.제�그�그�그�그����222222222222222!!!
🔸 Reference : 셋째 그릇을 받아 들었을 제
------
🔹 

🔹 Prediction: Has he been here?!!!!!!!!!
🔸 Reference : Has he been here?
------
🔹 Prediction: He also had a mountain made after him in the Antarctic.!!!!!!!!!
🔸 Reference : He also had a mountain named after him in the Antarctic.
------
🔹 Prediction: The family that moved to Chicago, arriving on the day, War I ended.!!!!!!!!!
🔸 Reference : The family later moved to Chicago, arriving on the day World War One ended.
------
🔹 Prediction: What is the benefit of the method in terms of identification accuracy?!!!!!!!!!
🔸 Reference : What is the benefit of the method in terms of identification accuracy?
------
🔹 Prediction: Thom's received a very of for his action during the kidnapping.!!!!!!!!!
🔸 Reference : Thompson received a bravery award for his actions during the kidnapping.
------
🔹 Prediction: The large house had hot watert.!!!!!!!!!
🔸 Reference : The large house had hot water taps.
------
🔹 Prediction: He spoke the words "Kolombia", Houston.!!!!!!!!!
🔸 Reference : He spoke the words 

🔹 Prediction: シルパーサーファー周�げ�点件我に、チャーズは地ームメと問に国際的にスーパー�ー�ーを�びゆ名人として認知されている!!!
🔸 Reference : シルバーサーファー襲撃事件までにリチャーズはチーム名と共に国際的にスーパーヒーローおよび有名人として認知されている
------
🔹 Prediction: あととえに�げらすみ小みはなくとの前前い一ちの場ゆいのようににに、体どのじでは言ておかななけれけれりません。!!!
🔸 Reference : たとえ逃げだす見こみはなくとも、まんいちのばあいの用意に、からだの自由だけは得ておかねばなりません。
------
🔹 Prediction: いいいじゃないかいいいいいいいいいいいいいややや、、、、、、、や、、、、やややややや、、ややや、、、、、!!!
🔸 Reference : 良いじゃないか。
------
🔹 Prediction: 設�対は�盾的一同一的現在として�自己を身を��すると�作的世界の�形を素知して、がが!!!
🔸 Reference : 絶対矛盾的自己同一的現在として、自己自身を形成する創造的世界の形成要素として、
------
🔹 Prediction: これ�が小かい人を先業をしてつに引き受けているがががが--ががががががやややややややややややや!!!
🔸 Reference : 彼が細かい事務作業を一手に引き受けている
------
🔹 Prediction: 先前生の任命に明たり対細ささ三、33333333333、11、、、、、、、、、、、、、ややややや、、、!!!
🔸 Reference : 最前線の任務にあたる
------
🔹 Prediction: おうものは何何もなかった。ををををををををををををををををををををををををををををををををを、、、、、、、、、!!!
🔸 Reference : 失う物は、何もなかった。
------
🔹 Prediction: 知識はする自身の声�やを問っている---がががも。。。。。。。。。。。。。。。。。。。。。!!!
🔸 Reference : 常識はそれ自身の効用をもっている。
------
🔹 Prediction: 世界は�限なあ��を持下わ々に乨み来るもののである。ももももむむむもむむむも!!!!!!!!!!

🔹 Prediction: Der Church back fließt durch den Ort, er in das Kn-mndet.!!!!!!!!!
🔸 Reference : Der Church Beck fließt durch den Ort bevor er in das Coniston Water mündet.
------
🔹 Prediction: In Gegenteil, sie kann sogar eine größer amreiz vor eine gewisse schleunigung sein.!!!!!!!!!
🔸 Reference : Im Gegenteil, sie kann sogar ein größerer Anreiz für eine gewisse Beschleunigung sein.
------
🔹 Prediction: Langsam berückkten sich die Zustände wieder und die Soldatenkhrten nach Hause zurück.!!!!!!!!!
🔸 Reference : Langsam beruhigten sich die Zustände wieder und die Soldaten kehrten nach Hause zurück.
------
🔹 Prediction: Allule Akteur sollten B Fide, Möglichkeiten nutzen, die die ganzesverhandlungen bieten.!!!!!!!!!
🔸 Reference : Alle Akteure sollten bona fide die Möglichkeiten nutzen, die die Genfer Verhandlungen bieten.
------
🔹 Prediction: Ein weiteres schwierigig Feld steht die Erennung hochrelicher Beter dar.!!!!!!!!!
🔸 Reference : Ein weiteres schwieriges Feld stellt die Ernennung ho

In [ ]:
print("\n✅✅✅ Language-wise Evaluation Results ✅✅✅")
for lang, scores in lang_scores.items():
    print(f"{lang.upper()} - WER: {scores['WER']:.4f} | CER: {scores['CER']:.4f}")


✅✅✅ Language-wise Evaluation Results ✅✅✅
KO - WER: 0.6065 | CER: 0.7682
EN - WER: 0.3687 | CER: 0.2754
JA - WER: 1.0100 | CER: 1.6870
DE - WER: 0.4724 | CER: 0.2690
